In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report
import pandas as pd
import time
import csv
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier

In [ ]:
class Report:

    header_csv = ["exec_time", "acc", "sen", "spe", "segment_size", "quantity_bins", "quantity_segments", "model"]
    filename_csv = 'results.csv'

    def __init__(self):
        self.start = time.time()
        self.end = None
        self.report_folds = []
        self.report_resume = None

    def register_report_fold(self, y, y_pred):
        report = classification_report(y, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=True, zero_division='warn')
        report_fold = [report['accuracy'], report['1.0']['recall'], report['0.0']['recall']]
        self.report_folds.append(report_fold)

    def register_report_resume(self):
        acc_resume = np.array([r[0] for r in self.report_folds])
        sen_resume = np.array([r[1] for r in self.report_folds])
        spe_resume = np.array([r[2] for r in self.report_folds])
        label_acc_resume = f"{acc_resume.mean().round(4)}+-{acc_resume.std().round(4)}"
        label_sen_resume = f"{sen_resume.mean().round(4)}+-{sen_resume.std().round(4)}"
        label_spe_resume = f"{spe_resume.mean().round(4)}+-{spe_resume.std().round(4)}"
        self.report_resume = [label_acc_resume, label_sen_resume, label_spe_resume]


    def persist_report_resume(self, segment_size, quantity_bins, quantity_segments, model):
        row_data = [
            time.strftime("%H:%M:%S", time.gmtime(self.end - self.start)),
            self.report_resume[0],
            self.report_resume[1],
            self.report_resume[2],
            segment_size,
            quantity_bins, 
            quantity_segments,
            str(model).replace('\n', '').replace('              ', '')]

        if not os.path.exists(self.filename_csv):
            with open(self.filename_csv, "w", newline="\n") as csvfile:
                csvwriter = csv.writer(csvfile, delimiter=';')
                csvwriter.writerow(self.header_csv)
        else:
            with open(self.filename_csv, "r", newline="\n") as csvfile:
                csvreader = csv.reader(csvfile, delimiter=';')
                if not any(row for row in csvreader):
                    with open(self.filename_csv, "w", newline="\n") as csvfile:
                        csvwriter = csv.writer(csvfile, delimiter=';')
                        csvwriter.writerow(self.header_csv)

        with open(self.filename_csv, "a", newline="\n") as csvfile:
            csvwriter = csv.writer(csvfile, delimiter=';')
            csvwriter.writerow(row_data)

    def get_df_results(self):
        report_full = self.report_folds[:]
        report_full.append(self.report_resume)
        indexes = list(range(1, 11))
        indexes.append('Average')
        return pd.DataFrame(report_full, columns=['Accuracy', 'Sensibility', 'Specificity'], index=indexes)

    def print_report(self):
        self.end = time.time()
        self.register_report_resume()
        return self.get_df_results()
        

In [ ]:
SEGMENT_SIZES = [10, 20, 30, 40, 50]
M_BINS_SIZES = [10, 20, 30, 40, 50]
NODES_NN = [(10, 10), (20, 20), (30, 30), (40, 40), (50, 50)]
RANGES_HIST = [None, (0, 5000), (50, 2500), (50, 2000)]

QUANTITY_SEGMENTS = 10000
DATABASES = ['AFDB', 'NSRDB', 'LTAFDB']

for SEGMENT_SIZE in SEGMENT_SIZES:

    for M_BINS in M_BINS_SIZES:

        for RANGE_HIST in RANGES_HIST:

            base = np.empty((0, M_BINS + 1))

            for db in DATABASES:
                label = 0 if db == 'NSRDB' else 1
                
                rri_output = np.load(f"output/rri/{db}.npy")

                rri_output = rri_output[0:QUANTITY_SEGMENTS*SEGMENT_SIZE]

                rri_size = rri_output.shape[0]
                
                segments = np.empty((0, SEGMENT_SIZE))
                for i in range(0, rri_size, SEGMENT_SIZE):
                    segment = rri_output[i:i+SEGMENT_SIZE]
                    if (segment.shape[0] == SEGMENT_SIZE):
                        segments = np.vstack((segments, segment))

                for segment in segments:
                    hist, _ = np.histogram(a=segment, range=RANGE_HIST, bins=M_BINS)
                    x = hist
                    y = label
                    row = np.append(x, y)
                    row = row.reshape((1, row.shape[0]))
                    base = np.vstack((base, row))

            for NODE_NN in NODES_NN:

                print(SEGMENT_SIZE, QUANTITY_SEGMENTS, RANGE_HIST, NODE_NN)

                model = MLPClassifier(
                            activation='tanh', 
                            alpha=2.6e-06, 
                            beta_1=0.999,
                            beta_2=0.9, 
                            hidden_layer_sizes=NODE_NN,
                            learning_rate_init=0.001, 
                            max_iter=64,
                            n_iter_no_change=32, 
                            random_state=1, 
                            validation_fraction=0.0,
                            verbose=0, warm_start=True)

                skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

                report = Report()

                y = base[:, -1]
                X = base[:, 0:-1]

                for train_index, test_index in skf.split(X, y):
                    x_train_fold, x_test_fold = X[train_index], X[test_index]
                    y_train_fold, y_test_fold = y[train_index], y[test_index]
                    model.fit(x_train_fold, y_train_fold)
                    y_pred = model.predict(x_test_fold)
                    report.register_report_fold(y=y_test_fold, y_pred=y_pred)

                print(report.report_folds[-1])
                report.print_report()
                report.persist_report_resume(SEGMENT_SIZE, M_BINS, QUANTITY_SEGMENTS, model)